In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from dotenv import load_dotenv

load_dotenv("../../.env.localhost")

In [ ]:
import warnings

import dagster

warnings.filterwarnings("ignore", category=dagster.ExperimentalWarning)

# Site Client Session

In [ ]:
from nmdc_runtime.util import nmdc_jsonschema, nmdc_jsonschema_validate

In [ ]:
from nmdc_runtime.site.repository import run_config_frozen__normal_env
from nmdc_runtime.site.resources import get_mongo


mongo = get_mongo(run_config_frozen__normal_env)
mdb = mongo.db

In [ ]:
from nmdc_runtime.site.repository import run_config_frozen__normal_env
from nmdc_runtime.site.resources import get_runtime_api_site_client


client = get_runtime_api_site_client(run_config_frozen__normal_env)

In [ ]:
# import os

# from nmdc_runtime.resources.core import RuntimeApiSiteClient

# client = RuntimeApiSiteClient(
#     base_url="https://drs.microbiomedata.org",
#     site_id=os.getenv("API_SITE_ID"),
#     client_id=os.getenv("API_SITE_CLIENT_ID"),
#     client_secret=os.getenv("API_SITE_CLIENT_SECRET"),
# )

# URL-accessible metadata files -> drs objects -> tagged -> validated -> ingested

In [ ]:
import re

from jsonschema import Draft7Validator
from nmdc_schema.validate_nmdc_json import get_nmdc_schema
import requests

from nmdc_runtime.util import nmdc_jsonschema

url_pattern = re.compile(r"https?://(?P<domain>[^/]+)/(?P<path>.+)")


def url_to_name(url):
    m = url_pattern.match(url)
    return f"{'.'.join(reversed(m.group('domain').split('.')))}__{m.group('path').replace('/', '.')}"


def result_for_url_to_json_file(response, url, save_dir):
    with open(os.path.join(save_dir, url_to_name(url)), "w") as f:
        json.dump(response.json(), f)


def fetch_downloaded_json(url, save_dir):
    with open(os.path.join(save_dir, url_to_name(url))) as f:
        return json.load(f)
    
type_collections = {
    f'nmdc:{spec["items"]["$ref"].split("/")[-1]}': collection_name
    for collection_name, spec in nmdc_jsonschema["properties"].items()
    if collection_name.endswith("_set")
}

def load_url(url, timeout):
    return requests.get(url, timeout=timeout)

In [ ]:
rv = requests.get("https://data.microbiomedata.org/data/nmdc:mga0wa96/annotation/annotations.json")

In [ ]:
rv = requests.get("https://data.microbiomedata.org/data/sizes.txt")

In [ ]:
sizes = rv.text.splitlines()

In [ ]:
strsizes = [line.split('\t')[0] for line in sizes]

In [ ]:
total_bytes = sum([int(s) for s in strsizes])

In [ ]:
total_bytes / (1000 * 1000)

In [ ]:
" ".join([u.replace("https://data.microbiomedata.org/data/", "") for u in urls])

In [ ]:
from json import JSONDecodeError
import tempfile
import concurrent.futures



def validate_json_urls(urls):
    validator = Draft7Validator(get_nmdc_schema())
    validation_errors = {}
    with tempfile.TemporaryDirectory() as temp_dir:

        with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
            future_to_url = {executor.submit(load_url, url, 5): url for url in urls}
            for future in concurrent.futures.as_completed(future_to_url):
                pbar.update(i)
                url = future_to_url[future]
                try:
                    response = future.result()
                except Exception as exc:
                    validation_errors[url] = f"Exception: {exc}"
                    continue
                if response.status_code != 200:
                    validation_errors[url] = {"status_code": response.status_code, "details": response.text}
                else:
                    try:
                        response.json()
                        result_for_url_to_json_file(response, url, temp_dir)
                    except JSONDecodeError as e:
                        validation_errors[url] = "Invalid JSON"
                    except Exception as exc:
                        validation_errors[url] = f"Could not write to file: {exc}"
                        
        pbar.close()

        for url in tqdm(set(urls) - set(validation_errors)):
            docs = fetch_downloaded_json(url, temp_dir)
            if "activity_set" in docs:
                for doc in docs["activity_set"]:
                    doc_type = doc["type"]
                    try:
                        collection_name = type_collections[doc_type]
                    except KeyError:
                        msg = (
                            f"activity_set doc {doc.get('id', '<id missing>')} "
                            f"has type {doc_type}, which is not in NMDC Schema. "
                            "Note: Case is sensitive."
                        )
                        if "activity_set" in validation_errors:
                            validation_errors["activity_set"].append(msg)
                        else:
                            validation_errors["activity_set"] = [msg]
                        continue

                    if collection_name in docs:
                        docs[collection_name].append(doc)
                    else:
                        docs[collection_name] = [doc]
                del docs["activity_set"]

            for coll_name, coll_docs in docs.items():
                errors = list(validator.iter_errors({coll_name: coll_docs}))
                validation_errors[coll_name] = [e.message for e in errors]
    
    return validation_errors

In [ ]:
from pathlib import Path

filepath = Path.home().joinpath(
    "Dropbox", "diary", "21", "09",
    "scanon-annotations-urls.txt"
)
with open(filepath) as f:
    urls = [line.strip() for line in f if line.strip()]

In [ ]:
validate_json_urls(urls)

# Validate Shane's results as nmdc metadata

In [ ]:
from pathlib import Path

filepath = Path.home().joinpath(
    "Dropbox", "diary", "21", "09",
    "2021-09-15-scanon-nmdc-metadata-file-urls.txt"
)
with open(filepath) as f:
    urls = [line.strip() for line in f if line.strip()]

In [ ]:
import json

def result_for_url_to_json_file(data, url):
        with open(f'/Users/dwinston/Downloads/{url_to_name(url)}', 'w') as f:
            json.dump(data.json(), f)
            
def fetch_downloaded_json(url):
    with open(f'/Users/dwinston/Downloads/{url_to_name(url)}') as f:
        return json.load(f)

In [ ]:
import concurrent.futures

from tqdm.notebook import tqdm
import requests

def load_url(url, timeout):
    return requests.get(url, timeout=timeout)

pbar = tqdm(total=len(urls))

with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    future_to_url = {executor.submit(load_url, url, 5): url for url in urls}
    for future in concurrent.futures.as_completed(future_to_url):
        url = future_to_url[future]
        try:
            data = future.result()
            result_for_url_to_json_file(data, url)
        except Exception as exc:
            print('%r generated an exception: %s' % (url, exc))
        else:
            print(f"saved {url} data")
        pbar.update(1)

pbar.close()

If `activity_set`, then read `type` field to get actual `*_set` to validate against.

In [ ]:
from nmdc_runtime.util import nmdc_jsonschema, nmdc_jsonschema_validate


type_collections = {
    f'nmdc:{spec["items"]["$ref"].split("/")[-1]}'.lower(): collection_name
    for collection_name, spec in
    nmdc_jsonschema["properties"].items()
    if collection_name.endswith("_set")
}

for url in tqdm(urls):
    docs = fetch_downloaded_json(url)
    if "activity_set" in docs:
        for doc in docs["activity_set"]:
            doc_type = doc["type"]
            collection_name = type_collections[doc_type.lower()]
            if collection_name in docs:
                docs[collection_name].append(doc)
            else:
                docs[collection_name] = [doc]
        del docs["activity_set"]
            
    _ = nmdc_jsonschema_validate(docs)

# Ensure no duplicate jobs

In [ ]:
from collections import defaultdict
from toolz import get_in

# TODO set unique =True index on [("workflow.id", 1), "config.object_id", 1)] for jobs collection?
#   Doesn't seem right, as "config" may have other key-value pairs.

jobs = defaultdict(list)

for doc in mdb.jobs.find():
    wf_id = get_in(["workflow", "id"], doc)
    config_object_id = get_in(["config", "object_id"], doc)
    jobs[(wf_id, config_object_id)].append(doc["id"])

assert all(len(instances) == 1 for key, instances in jobs.items())

# aggregated file counts and sizes by data type

In [ ]:
ids = [d["_id"] for d in mdb.data_object_set.find({"data_object_type": {"$exists": True}}, ["_id"])]

stats_docs = list(mdb.data_object_set.aggregate([
    {"$match": {"data_object_type": {"$exists": True}}},
    {"$group": {
        "_id": "$data_object_type",
        "size_total": {"$sum": "$file_size_bytes"},
        "count": {"$sum": 1}
    }},
]))

for sdoc in stats_docs:
    edoc = mdb.file_type_enum.find_one({"id": sdoc["_id"]})
    sdoc["name"] = edoc["name"]
    sdoc["description"] = edoc["description"]
    
for d in stats_docs:
    print("{name} ({description}):".format(**d))
    print(f"total size (TB): {(d['size_total'] / (1024 * 1024 * 1024 * 1024)):.3}")
    print("count: {count}".format(**d))
    print()

# Manage User/Client Credentials

In [ ]:
from nmdc_runtime.api.core.auth import get_password_hash

username = ""
password = ""

rv = mdb.users.replace_one(
    {"username": username},
    {"username": username, "hashed_password": get_password_hash(password)},
    upsert=True
)
rv

# Test operations

In [ ]:
from datetime import datetime, timezone, timedelta

from nmdc_runtime.api.core.idgen import generate_one_id
from nmdc_runtime.api.models.operation import Operation

dt_past = datetime.now(timezone.utc) - timedelta(days=1)

mdb.operations.insert_one(Operation(id=generate_one_id(mdb, "op"), expire_time=dt_past).dict(exclude_unset=True))

# demo e2e workflow job creation via sensor, job claiming, and operation updating to done=true

example for demo: metaP metadata ETL

### register objects for existing URLs

In [ ]:
import os
import json
from pathlib import Path
import re
from tempfile import TemporaryDirectory

import requests

from nmdc_runtime.util import drs_metadata_for

pattern = re.compile(r"https?://(?P<domain>[^/]+)/(?P<path>.+)")

def url_to_name(url):
    m = pattern.match(url)
    return f"{'.'.join(reversed(m.group('domain').split('.')))}__{m.group('path').replace('/', '.')}"

def download_them_all(to_fetch, dirname):
    results = []
    for i, spec in enumerate(to_fetch):
        url = spec["url"]
        name = url_to_name(url)
        print(f"{i+1}/{len(to_fetch)}: fetching {url}")
        rv = requests.get(url)
        filename = os.path.join(dirname, name)
        print(f"saving as {filename}")
        with open(filename, "w") as f:
            json.dump(rv.json(), f)
        results.append(dict(**spec, filename=filename))
    return results

def generate_drsobject_metadata_for(to_fetch):
    with TemporaryDirectory() as dirname:
        to_fetch = download_them_all(to_fetch, dirname)

        drsobjects = []
        for i, spec in enumerate(to_fetch):
            base = {
                "access_methods": [{"access_url": {"url": spec["url"]}}],
                "description": spec["type"],
            }
            print(f"{i+1}/{len(to_fetch)}: generating DrsObject metdata for {spec['url']}")
            drsobjects.append(drs_metadata_for(spec["filename"], base=base))
        return drsobjects

In [ ]:
to_fetch = [{
    # >100MB
    "url": "https://portal.nersc.gov/cfs/m3408/meta/stegen_MetaProteomicAnalysis_activity.json",
    "type": "metaproteomics_analysis_activity_set",
}, {
    # ~50KB
    "url": "https://portal.nersc.gov/cfs/m3408/meta/stegen_emsl_analysis_data_objects.json",
    "type": "data_object_set"
}]

In [ ]:
drsobjects = generate_drsobject_metadata_for(to_fetch)

In [ ]:
from nmdc_runtime.api.models.object import DrsObjectIn

for o in drsobjects:
    DrsObjectIn(**o)

In [ ]:
def json_clean(d, model, exclude_unset=False):
    return json.loads(model(**d).json(exclude_unset=exclude_unset))

In [ ]:
json_clean(drsobjects[0], DrsObjectIn)

In [ ]:
rv = client.create_object(json_clean(drsobjects[0], DrsObjectIn))
rv.content

In [ ]:
rv.json()

In [ ]:
rv = client.create_object(json_clean(drsobjects[1], DrsObjectIn))

In [ ]:
rv.json()

In [ ]:
mdb.objects.find_one({"name": "gov.nersc.portal__cfs.m3408.meta.stegen_MetaProteomicAnalysis_activity.json"})

### register object_types for objects

In [ ]:
object_id = "sys0b109"
rv = client.request("PUT", f"/objects/{object_id}/types", ["metaproteomics_analysis_activity_set"])
rv.content

In [ ]:
rv.json()

### confirm sensor logic for creating target job, and enable sensor in dagit
- [x] sensor: metaproteomics_analysis_activity_ingest
- [x] sensor yields run request for configured graph: ensure_job

### job claiming

In [ ]:
# TODO

rv = client.request("GET", "/jobs")
rv.json()

In [ ]:
job_id = ""
rv = client.claim_job(job_id)

In [ ]:
rv.json()

### operation updating to done=true

In [ ]:
rv = client.request("GET", "/operations")

In [ ]:
rv.json()

In [ ]:
# TODO

# register mongoexports as DrsObjects

The mongoexports at https://portal.nersc.gov/project/m3408/meta/mongoexports/2021-07/

In [ ]:
from pprint import pprint

nmdc_db_collection_names = set(nmdc_jsonschema["definitions"]["Database"]["properties"])
nmdc_db_collection_names -= {
    # not actually collections
    "activity_set",
    "nmdc_schema_version",
    "date_created",
    "etl_software_version",
}

pprint(nmdc_db_collection_names)

```bash
scp -r dtn01.nersc.gov:/project/projectdirs/m3408/www/meta/mongoexports/2021-07/ \
    /Users/dwinston/nmdc_files/mongoexports/
```

In [ ]:
from glob import glob

from nmdc_runtime.util import drs_metadata_for
import requests
from tqdm.notebook import tqdm

paths = glob(os.path.expanduser("~/nmdc_files/mongoexports/2021-07/*"))
url_base = "https://portal.nersc.gov/project/m3408/meta/mongoexports/2021-07"
objects = []
for path in tqdm(paths):
    meta = drs_metadata_for(path)
    url = f'{url_base}/{meta["name"]}'
    
    rv = requests.head(
        url, allow_redirects=True, timeout=5, headers={"Accept-Encoding": "gzip;q=0"}
    )
    if not rv.status_code == 200:
        raise Exception(f"url {url} not OK")

    try:
        size_bytes = int(rv.headers['Content-Length'])
    except KeyError:
        raise Exception(f"no content-length response for {url}")
        
    if size_bytes != meta["size"]:
        raise Exception(f'size of local file {path} ({meta["size"]}) does not match size at {url} ({size_bytes})')
    
    objects.append(DrsObjectIn(
        access_methods=[{"access_url": {"url": url}}],
        **meta
    ))

In [ ]:
rvs = []
for doc in tqdm([json.loads(o.json(exclude_unset=True)) for o in objects]):
    rvs.append(client.create_object(doc))

# Register a bundle

In [ ]:
# checksum computed over sorted concatenation of the corresponding checksums of its top-level contents
# size is cumulative size of contents
# created_time must be >= max of contents created_time values

from glob import glob
import os
from pathlib import Path

from toolz import dissoc

paths = glob(os.path.expanduser("~/mongoexport/2021-10-14/*.jsonl.gz"))
names = [Path(p).name for p in paths]
bundle_content_objects = [
    dissoc(mdb.objects.find_one(
        filter={"name": n},
        projection=["id", "name", "size", "checksums", "created_time"],
        sort=[("created_time", -1)]
    ), "_id")
    for n in names
]

In [ ]:
bundle_content_objects

In [ ]:
len(checksums)

In [ ]:
checksums = [
    c["checksum"]
    for bco in bundle_content_objects
    for c in bco["checksums"]
    if c["type"] == "sha256"
]
if len(checksums) != len(bundle_content_objects):
    raise Exception("Contents of bundle must have sha-256 checksums to compute bundle checksum")

In [ ]:
from nmdc_runtime.api.core.util import hash_from_str

checksum = hash_from_str("".join(sorted(checksums)), algo="sha256")

In [ ]:
size = sum(o["size"] for o in bundle_content_objects)

In [ ]:
created_time = max(o["created_time"] for o in bundle_content_objects)

In [ ]:
checksum, size, created_time

In [ ]:
from nmdc_runtime.api.core.util import pick
from nmdc_runtime.api.models.object import DrsObjectIn

drs_bundle_in = DrsObjectIn(
    name="2021-10-nmdcdb-mongoexport",
    description="bgzipped json-lines mongoexports of nmdc:Database collections on 2021-10-14",
    checksums=[{"type": "sha256", "checksum": checksum}],
    contents=[pick(["id", "name"], o) for o in bundle_content_objects],
    created_time=created_time,
    size=size,
)

In [ ]:
import json
from pprint import pprint

pprint(json.loads(drs_bundle_in.json(exclude_unset=True)))

In [ ]:
rv = client.create_object(json.loads(drs_bundle_in.json(exclude_unset=True)))

In [ ]:
rv.json()

# Register objects by S3 upload and access IDs

In [ ]:
!echo '{"hello": "donny"}' > test.json

In [ ]:
from pathlib import Path

from dagster import build_solid_context
from nmdc_runtime.site.ops import local_file_to_api_object as lftao

context = build_solid_context(
    resources={"mongo": mongo, "runtime_api_site_client": client}
)

storage_path = "test.json"


def local_file_to_api_object(file_info):
    return lftao(context, file_info)

obj = next(local_file_to_api_object({"storage_path": storage_path, "mime_type": 'application/json'}))

doc = mdb.objects.find_one({"id": obj["id"]})
assert doc["name"] == Path(storage_path).name

# Upload object and let sensor-triggered pipeline register object

In [ ]:
!echo '{"hello": "donny"}' > test.json

In [ ]:
rv = client.put_object_in_site({"mime_type": "application/json", "name": "test.json"})

In [ ]:
op = rv.json()

In [ ]:
op

In [ ]:
from nmdc_runtime.util import put_object

rv = put_object("test.json", op["metadata"]["url"])
rv

In [ ]:
from nmdc_runtime.util import drs_object_in_for

result = drs_object_in_for("test.json", op)

In [ ]:
result

In [ ]:
import json
from pprint import pprint

op_patch = {"done": True, "result": result}

In [ ]:
rv = client.update_operation(op["id"], op_patch)

In [ ]:
pprint(rv.json())

# S3 init

In [ ]:
from functools import lru_cache
import os

import boto3

API_SITE_BUCKET = os.getenv("API_SITE_ID")


@lru_cache
def get_s3_client():
    _session = boto3.session.Session()
    return _session.client(
        "s3",
        region_name=os.getenv("DO_REGION_NAME"),
        endpoint_url=os.getenv("DO_ENDPOINT_URL"),
        aws_access_key_id=os.getenv("DO_SPACES_KEY"),
        aws_secret_access_key=os.getenv("DO_SPACES_SECRET"),
    )


def presigned_url_to_put(
    key, client=None, mime_type=None, bucket=API_SITE_BUCKET, expires_in=300
):
    return client.generate_presigned_url(
        ClientMethod="put_object",
        Params={"Bucket": bucket, "Key": key, "ContentType": mime_type},
        ExpiresIn=expires_in,
    )

s3client = get_s3_client()

response = s3client.list_buckets()
for space in response['Buckets']:
    print(space['Name'])
print(f"\nusing Bucket {API_SITE_BUCKET}\n")
response = s3client.list_objects(Bucket=API_SITE_BUCKET)
for obj in response['Contents']:
    print(obj['Key'])

# GSP schema  / Cordra stuff

Need "id" in payload, e.g.:
```
"results": [
    {
      "id": "test/activity",
      "type": "Schema",
      "content": {
        "name": "Activity",
        "schema": collschemas["activity_set"]
      }
    }
  ]
```

In [ ]:
from time import time
import os

tic = time()

from dotenv import load_dotenv
load_dotenv(os.path.expanduser("~/.nmdc_mongo.env"))

In [ ]:
os.environ["NMDC_JSON_SCHEMA_FILE"] = "/Users/dwinston/Desktop/nmdc.schema.gsp.json"

In [ ]:
import json
import re
from toolz import assoc_in, dissoc
from zipfile import ZipFile

from mongospawn.schema import collschemas_for

from nmdc_mongo import (
    add_to_db,
    correct_metaP_doc,
    dbschema,
    fetch_and_validate_json,
    fetch_conform_and_persist_from_manifest,
    fetch_json,
    get_db,
    reset_database,
    snake_case_set_name
)

In [ ]:
###########################
# Adjustments for GSP below
###########################

defined_object_names = set(dbschema["definitions"])

set_for_object_name = {
    spec["items"]["$ref"].split("#/definitions/")[-1]: set_name
    for set_name, spec in dbschema["properties"].items()
}

existing_set_names = set(dbschema["properties"])

for object_without_set in (defined_object_names - set(set_for_object_name.keys())):
    proposed_set_name = snake_case_set_name(object_without_set)
    if proposed_set_name not in existing_set_names:
        dbschema["properties"][proposed_set_name] = {
            "description": (f"This property links a database object to the set of"
                            f" {object_without_set} objects within it."),
            "items": {"$ref": f"#/definitions/{object_without_set}"},
            "type": "array",
        }
        
dbschema = assoc_in(dbschema, ["definitions", "ControlledTermValue", "properties", "term", "type"], "string")
del dbschema["definitions"]["ControlledTermValue"]["properties"]["term"]["$ref"]

# 'k' not capitalized upstream perhaps. should conform!
#dbschema = assoc_in(dbschema, ["definitions", "MetagenomeAssembly", "properties", "scaf_l_gt50k", "type"], "number")

In [ ]:
collschemas = collschemas_for(dbschema)

# Reconstruct
set_for_object_name = {
    spec["items"]["$ref"].split("#/definitions/")[-1]: set_name
    for set_name, spec in dbschema["properties"].items()
}

In [ ]:
sorted(collschemas.keys())

In [ ]:
collschemas["biosample_set"]

In [ ]:
import requests

In [ ]:
rv = requests.post("http://localhost:8080/auth/token",
              {"grant_type": "password", "username": "", "password": ""})

In [ ]:
rv.json()

In [ ]:
auth_header = {"Authorization": f'Bearer {rv.json()["access_token"]}'}

In [ ]:
auth_header

In [ ]:
rv = requests.post("http://localhost:8080/uploadObjects", json={
    "results": [
        {
          "type": "Schema",
          "content": {
            "name": "Biosample",
            "schema": collschemas["biosample_set"]
          }
        }
      ]
}, headers=auth_header)

In [ ]:
rv

In [ ]:
import json
with open('/Users/dwinston/Desktop/cordra-upload.json','w') as f:
    json.dump({
        "results": [
            {
              "id": "test/activity",
              "type": "Schema",
              "content": {
                "name": "Activity",
                "schema": collschemas["activity_set"]
              }
            }
          ]
    }, f, indent=2)

In [ ]:
import json
with open('/Users/dwinston/Desktop/cordra-upload.json','w') as f:
    json.dump({
        "results": [
            {
              "id": "test/study",
              "type": "Schema",
              "content": {
                "name": "Study",
                "schema": collschemas["study_set"]
              }
            }
          ]
    }, f, indent=2)

In [ ]:
rv = requests.get("http://localhost:8080/search?query=type:%22Schema%22", headers=auth_header)

In [ ]:
from pprint import pprint

template = rv.json()
del template["pageNum"]
del template["pageSize"]
del template["size"]
del template["results"][0]
del template["results"][0]
template["results"][0]["id"] = "test/abcd1234"
template["results"][0]["content"]["name"] = "Document2"
template["results"][0]["content"]["schema"]["title"] = "Document2"
del template["results"][0]["content"]["identifier"]

pprint(template)

In [ ]:
import json
with open('/Users/dwinston/Desktop/cordra-upload.json','w') as f:
    json.dump(template, f, indent=2)

In [ ]:
!cat /Users/dwinston/Desktop/cordra-upload.json

In [ ]:
rv = requests.post("http://localhost:8080/uploadObjects", json=template, headers=auth_header)
rv